In [2]:
from DeePyMoD_SBL.data.diffusion import library
from DeePyMoD_SBL.SBL import SBL
from DeePyMoD_SBL.SBL import robust_SBL

import numpy as np
import seaborn as sns

%load_ext autoreload
%autoreload 2

# Making test data

In [2]:
x_points = 100
t_points = 25
x_grid, t_grid = np.meshgrid(np.linspace(-10, 10, x_points), np.linspace(0, 1, t_points), indexing='ij')
time_deriv, theta = library(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1))

In [3]:
level = 0.0001
y = time_deriv + np.random.normal(scale=level*np.std(time_deriv), size=time_deriv.shape)

In [115]:
from DeePyMoD_SBL.SBL import robust_SBL
from DeePyMoD_SBL.data import Burgers

In [116]:
x_points = 100
t_points = 25
x_grid, t_grid = np.meshgrid(np.linspace(-10, 10, x_points), np.linspace(1e-4, 1, t_points), indexing='ij')

In [117]:
burgers = Burgers(0.1, 1)
theta = burgers.library(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1))
time_deriv = burgers.time_deriv(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1))

In [118]:
level = 0.5
y = time_deriv + level*np.std(time_deriv)* np.random.normal(size=time_deriv.shape)

# Developinh

In [119]:
from copy import copy

In [146]:
def posterior(Phi, gamma, y):
    Sigma = np.linalg.inv(Phi.T @ Phi + np.diag(gamma[gamma != 0.0]**-1))
    mu = Sigma @ Phi.T @ y
    
    return mu, Sigma

In [147]:
def sparse_quality_factor(theta, Phi, Sigma, gamma, y):
    precalc = Phi @ Sigma @ Phi.T

    Sm = np.concatenate([phi_i[:, None].T @ phi_i[:, None] - phi_i[:, None].T @ precalc @ phi_i[:, None] for phi_i in theta.T])
    Qm = np.concatenate([phi_i[:, None].T @ y - phi_i[:, None].T @ precalc @ y for phi_i in theta.T])
    
    si = Sm / (1 - gamma[:, None] * Sm)
    qi = Qm / (1 - gamma[:, None] * Sm)

    return si, qi

In [148]:
def update_design_matrix(Phi, theta, gamma, si, qi, beta, l, idx=None):
    g = copy(gamma)
    n_terms = theta.shape[1]
    if idx == None:
        idx = np.random.choice(n_terms)
    criterium  = beta * qi[idx]**2 - si[idx]
    
    if criterium > l:
        g[idx] = (-2 * l - si[idx] + np.sqrt(4*beta*l*qi[idx]**2+si[idx]**2)) / (2*l*si[idx])
    else:
        g[idx] = 0
        
    Phi = theta[:, g != 0.0]
    
    return Phi, g

In [149]:
def update_noise(N, y, Phi, mu):
    beta = N * np.linalg.inv(y.T @ (y - Phi @ mu))
    return beta

In [154]:
def update_l(l, nu, delta, gamma, n_samples):
    delta = (n_samples + nu) / np.max(beta * qi**2 - si)
    l = 2 * (n_samples - 1 + nu) / (np.sum(gamma) + 2 * delta)
    return l

In [163]:
n_terms = theta.shape[1]
n_samples = theta.shape[0]
    
theta_normed = theta / np.linalg.norm(theta, axis=0, keepdims=True)
y_normed = y / np.linalg.norm(y)

gamma = np.ones((n_terms))
Phi = theta_normed

beta = 1 / (0.1 * np.var(y_normed))
l = 0.0


In [169]:
mu, Sigma = posterior(Phi, gamma, y_normed)
si, qi = sparse_quality_factor(theta_normed, Phi, Sigma, gamma, y_normed)

In [177]:
beta * qi**2 - si

array([[   2.47242009],
       [1944.55109328],
       [2207.52510524],
       [1157.37064008],
       [   4.18904499],
       [ 827.49954375],
       [ 507.56473478],
       [   5.9998539 ],
       [ 136.26524312],
       [ 479.09341174],
       [  55.02926196],
       [ 216.76782494]])

In [157]:
its = 1000
for iteration in np.arange(its):
    Phi, gamma = update_design_matrix(Phi, theta_normed, gamma, si, qi, beta, l)
    mu, Sigma = posterior(Phi, gamma, y_normed)
    si, qi = sparse_quality_factor(theta_normed, Phi, Sigma, gamma, y_normed)
    beta = update_noise(n_samples, y_normed, Phi, mu)
    l = update_l(l, nu, delta, gamma, n_samples)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [158]:
gamma

array([0.        , 0.        , 0.00022894, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])

In [159]:
l

1268.658357966714

In [160]:
beta * qi**2 - si

array([[-9.97526386e-01],
       [ 1.12880100e+03],
       [ 1.26923955e+03],
       [ 4.91654553e+02],
       [ 1.03883567e+02],
       [ 7.50984383e+02],
       [ 5.30385268e+02],
       [ 1.33153192e+01],
       [ 3.68439685e+02],
       [ 4.96093662e+02],
       [ 1.68150500e+02],
       [ 3.63624488e+02]])

In [162]:
beta * qi **2 - si

array([[-9.97526386e-01],
       [ 1.12880100e+03],
       [ 1.26923955e+03],
       [ 4.91654553e+02],
       [ 1.03883567e+02],
       [ 7.50984383e+02],
       [ 5.30385268e+02],
       [ 1.33153192e+01],
       [ 3.68439685e+02],
       [ 4.96093662e+02],
       [ 1.68150500e+02],
       [ 3.63624488e+02]])

In [10]:
def robust_SBL(theta, y, its=100, nu=1.0, delta=5.0):
    n_terms = theta.shape[1]
    n_samples = theta.shape[0]
    
    theta_normed = theta / np.linalg.norm(theta, axis=0, keepdims=True)
    y_normed = y / np.linalg.norm(y)

    gamma = np.ones((n_terms))
    Phi = theta_normed

    beta = 1 / (0.1 * np.var(y_normed))
    l = 0.0
    
    mu, Sigma = posterior(Phi, gamma, y_normed)
    si, qi = sparse_quality_factor(theta_normed, Phi, Sigma, gamma, y_normed)
    for iteration in np.arange(its):
        Phi, gamma = update_design_matrix(Phi, theta_normed, gamma, si, qi, beta, l)
        mu, Sigma = posterior(Phi, gamma, y_normed)
        si, qi = sparse_quality_factor(theta_normed, Phi, Sigma, gamma, y_normed)
        l = update_l(l, nu, delta, gamma, n_samples):
        beta = update_noise(n_samples, y_normed, Phi, mu)
    return gamma

In [11]:
robust_SBL(theta, t)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


array([0.        , 0.        , 7.39955183, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])

# Adding initialization

In [56]:
def initialize(theta, y):
    theta_normed = theta / np.linalg.norm(theta, axis=0, keepdims=True)
    y_normed = y / np.linalg.norm(y)
    beta = 1 / (0.1 * np.var(y_normed))
    
    ini_idx = np.argmax((theta_normed.T @ y_normed)**2)
    Phi = theta_normed[:, [ini_idx]]
    n_terms = theta.shape[1]
    gamma = np.zeros(n_terms)
    gamma[ini_idx] = beta * (Phi.T @ y_normed)**2 - 1
    
    return beta, gamma, Phi, theta_normed, y_normed

In [61]:
def robust_SBL(theta, y, its=100, nu=1.0, delta=5.0):
    beta, gamma, Phi, theta_normed, y_normed = initialize(theta, y)
    
    l = 0.0
    n_samples = theta_normed.shape[0]
    
    mu, Sigma = posterior(Phi, gamma, y_normed)
    si, qi = sparse_quality_factor(theta_normed, Phi, Sigma, gamma, y_normed)
    for iteration in np.arange(its):
        Phi, gamma = update_design_matrix(Phi, theta_normed, gamma, si, qi, beta, l)
        mu, Sigma = posterior(Phi, gamma, y_normed)
        si, qi = sparse_quality_factor(theta_normed, Phi, Sigma, gamma, y_normed)
        l = update_l(l, nu, delta, gamma, n_samples)
        beta = update_noise(n_samples, y_normed, Phi, mu)
    return gamma

In [62]:
level = 0.1
y = time_deriv + np.random.normal(scale=level*np.std(time_deriv), size=time_deriv.shape)

In [64]:
robust_SBL(theta, y, its=1000)

array([0.        , 0.        , 6.83932541, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])

In [ ]:
robust_SBL(theta, y, its=10000)

# Convergence

In [127]:
def convergence(si, qi, l, beta, gamma):
    in_model_idx = beta * qi**2 - si > l
    
    dgamma = gamma[in_model_idx.squeeze()] - ((-2 * l - si[in_model_idx] + np.sqrt(4*beta*l*qi[in_model_idx]**2+si[in_model_idx]**2)) / (2*l*si[in_model_idx])).squeeze()
    converged = np.max(np.abs(dgamma)) < 1e-6
    
    return converged

In [128]:
level = 0.1
y = time_deriv + np.random.normal(scale=level*np.std(time_deriv), size=time_deriv.shape)

its = 100

delta = 5.0
nu = 1.0

In [129]:
beta, gamma, Phi, theta_normed, y_normed = initialize(theta, y)
    
l = 0.0
n_samples = theta_normed.shape[0]
converged = False    
    
mu, Sigma = posterior(Phi, gamma, y_normed)
si, qi = sparse_quality_factor(theta_normed, Phi, Sigma, gamma, y_normed)
while not converged:
    Phi, gamma = update_design_matrix(Phi, theta_normed, gamma, si, qi, beta, l)
    mu, Sigma = posterior(Phi, gamma, y_normed)
    si, qi = sparse_quality_factor(theta_normed, Phi, Sigma, gamma, y_normed)
    l = update_l(l, nu, delta, gamma, n_samples)
    beta = update_noise(n_samples, y_normed, Phi, mu)
    converged = convergence(si, qi, l, beta, gamma)

In [131]:
gamma

array([0.        , 0.        , 6.78832938, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])

# Testing

In [4]:
x_points = 100
t_points = 25
x_grid, t_grid = np.meshgrid(np.linspace(-10, 10, x_points), np.linspace(0, 1, t_points), indexing='ij')
time_deriv, theta = library(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1))

In [5]:
level = 2.
y = time_deriv + np.random.normal(scale=level*np.std(time_deriv), size=time_deriv.shape)

In [6]:
robust_SBL(theta, y, delta=10.0)

array([0.        , 0.        , 0.48859183, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])

# Implementing optimal vec update

In [10]:
from copy import copy

In [2]:
x_points = 100
from DeePyMoD_SBL.SBL.robust_SBL import initialize, posterior, sparse_quality_factor, update_design_matrix, update_l, update_noise, convergence
t_points = 25
x_grid, t_grid = np.meshgrid(np.linspace(-10, 10, x_points), np.linspace(0, 1, t_points), indexing='ij')
time_deriv, theta = library(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1))

In [3]:
level = 0.5
y = time_deriv + np.random.normal(scale=level*np.std(time_deriv), size=time_deriv.shape)

In [4]:
delta = 5.0
nu = 1.0
l = 0.0
n_samples = theta.shape[0]


In [7]:
beta, gamma, Phi, theta_normed, y_normed = initialize(theta, y)
mu, Sigma = posterior(Phi, gamma, y_normed)
si, qi = sparse_quality_factor(theta_normed, Phi, Sigma, gamma, y_normed)

In [8]:
def dL(gamma, si, qi, beta, l):
    delta_l = -np.log(1 + gamma[:, None] * si) + beta * gamma[:, None] * qi**2/(1+gamma[:, None]*si) - l * gamma[:, None]
    return delta_l

In [14]:
gamma_current = copy(gamma)
gamma_new = ((-2 * l - si + np.sqrt(4*beta*l*qi**2+si**2)) / (2*l*si)).squeeze()

In [15]:
gamma_current

array([    0.        ,     0.        , 19672.39476688,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ])

In [16]:
gamma_new

array([-0.95752576, -0.99100694,  5.94266594, -0.96982041, -1.13909256,
       -0.97881221, -4.42046374, -0.95836079, -1.90146359, -0.96528824,
       -2.74138073, -0.952636  ])

In [17]:
in_basis_idx = (gamma_current != 0.0)
out_basis_idx = (gamma_current == 0.0)
allowed = beta * qi**2 - si > l

In [19]:
dL_adding = dL(gamma_new, si, qi, beta, l)
dL_adding[in_basis_idx] = 0.0
dL_adding[~allowed] = 0.0
    
dL_removing = -dL(gamma_current, si, qi, beta, l)
dL_removing[out_basis_idx] = 0.0
dL_removing[allowed] = 0.0
    
dL_updating = dL(gamma_new, si, qi, beta, l) -dL(gamma_current, si, qi, beta, l)
dL_updating[out_basis_idx] = 0.0
dL_updating[~allowed] = 0.0

In [20]:
dL_complete = np.concatenate((dL_adding, dL_updating, dL_removing), axis=1)
idx, action = np.unravel_index(np.argmax(dL_complete), dL_complete.shape)


In [69]:
if action == 0:
    gamma_current[idx] = gamma_new[idx]
elif action == 1:
    gamma_current[idx] = gamma_new[idx]
elif action == 2:
    gamma_current[idx] = 0.0


## Testing

In [83]:
import numpy as np

In [90]:
from DeePyMoD_SBL.SBL.robust_SBL import initialize, posterior, sparse_quality_factor, update_l, update_noise, convergence, update_design_matrix

In [91]:
x_points = 100
t_points = 25
x_grid, t_grid = np.meshgrid(np.linspace(-10, 10, x_points), np.linspace(0, 1, t_points), indexing='ij')
time_deriv, theta = library(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1))

In [98]:
level = 0.5
y = time_deriv + np.random.normal(scale=level*np.std(time_deriv), size=time_deriv.shape)

In [99]:
delta = 5.0
nu = 1.0
l = 1e-8
n_samples = theta.shape[0]


In [100]:
beta, gamma, Phi, theta_normed, y_normed = initialize(theta, y)
mu, Sigma = posterior(Phi, gamma, y_normed)
si, qi = sparse_quality_factor(theta_normed, Phi, Sigma, gamma, y_normed)
l = update_l(l, nu, delta, gamma, n_samples)

In [101]:
converged = False 
while not converged:
    Phi, gamma = update_design_matrix(Phi, theta_normed, gamma, si, qi, beta, l)
    mu, Sigma = posterior(Phi, gamma, y_normed)
    si, qi = sparse_quality_factor(theta_normed, Phi, Sigma, gamma, y_normed)
    l = update_l(l, nu, delta, gamma, n_samples)
    beta = update_noise(n_samples, y_normed, Phi, mu)
    converged = convergence(si, qi, l, beta, gamma)

In [104]:
gamma

array([0.        , 0.        , 2.42527691, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])

# Updating nu

In [877]:
import numpy as np
from scipy.optimize import root_scalar
from scipy.special import digamma

In [878]:
from DeePyMoD_SBL.SBL.robust_SBL import initialize, posterior, sparse_quality_factor, update_l, update_noise, convergence, update_design_matrix

In [956]:
x_points = 100
t_points = 25
x_grid, t_grid = np.meshgrid(np.linspace(-10, 10, x_points), np.linspace(0, 1, t_points), indexing='ij')
time_deriv, theta = library(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1))

In [957]:
level = 0.5
y = time_deriv + np.random.normal(scale=level*np.std(time_deriv), size=time_deriv.shape)

In [979]:
1/0.14**2

51.0204081632653

In [958]:
delta = 5.0
nu = 0.0
l = 1e-4
n_samples = theta.shape[0]


In [1010]:
beta, gamma, Phi, theta_normed, y_normed = initialize(theta, y)
mu, Sigma = posterior(Phi, gamma, y_normed)
si, qi = sparse_quality_factor(theta_normed, Phi, Sigma, gamma, y_normed)

In [1011]:
beta

25000.10525077745

In [1012]:
beta * qi**2 - si

array([[-8.94766970e-01],
       [-7.95782762e-01],
       [ 1.99343598e+04],
       [ 5.22065743e-01],
       [ 3.32497168e-01],
       [ 1.19652652e+00],
       [-2.00298425e-01],
       [ 4.58137756e+00],
       [ 2.99751120e-01],
       [ 4.12925113e+00],
       [-3.13625906e-01],
       [ 6.09278429e+00]])

In [1013]:
np.max(gamma[gamma == 0.0])

0.0

In [1014]:
l = update_l(l, nu, gamma, n_samples, gamma)
#beta = update_noise(n_samples, y_normed, Phi, mu)
#print(l)

#nu = root_scalar(lambda x: np.log(x/2) - digamma(x/2)  + 1 + np.log(l) - l, bracket=[1e-8, 1e3]).root
#print(nu)

In [1023]:
Phi, gamma, gamma_old = update_design_matrix(Phi, theta_normed, gamma, si, qi, beta, l)
mu, Sigma = posterior(Phi, gamma, y_normed)
si, qi = sparse_quality_factor(theta_normed, Phi, Sigma, gamma, y_normed)
l = update_l(l, nu, gamma, n_samples, gamma_old)
#nu = root_scalar(lambda x: np.log(x/2) - digamma(x/2)  + 1 + np.log(l) - l, bracket=[1e-8, 1e3]).root
beta = update_noise(n_samples, y_normed, Phi, mu)

In [1024]:
qi

array([[-0.0020517 ],
       [ 0.00285809],
       [ 0.89297893],
       [-0.00780272],
       [-0.08602986],
       [ 0.0093734 ],
       [ 0.1687064 ],
       [-0.01494169],
       [-0.14112759],
       [ 0.01432373],
       [ 0.15292493],
       [-0.0168437 ]])

In [908]:
si

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [909]:
l

20433.202926321283

In [918]:
beta * qi**2 - si - l

array([[-20434.15051341],
       [-20434.08087109],
       [-18435.43622663],
       [-20434.20248714],
       [-20091.61242607],
       [-20434.18729803],
       [-18859.19368119],
       [-20434.1860586 ],
       [-19423.84227658],
       [-20434.2017621 ],
       [-19124.60408899],
       [-20434.15748554]])

In [920]:
beta *qi**2

array([[5.24129081e-02],
       [1.22055232e-01],
       [1.99876670e+03],
       [4.39183730e-04],
       [3.42590500e+02],
       [1.56282866e-02],
       [1.57500925e+03],
       [1.68677203e-02],
       [1.01036065e+03],
       [1.16421972e-03],
       [1.30959884e+03],
       [4.54407824e-02]])

In [923]:
2*n_samples/np.sum(gamma_old)

10220.689739056264

In [465]:
root_scalar(lambda x: np.log(x/2) - digamma(x/2)  + 1 + np.log(l) - l, bracket=[1e-8, 1e3]).root

0.002404787728527145

In [329]:
root_scalar(lambda x: np.log((2*(n_samples -1) - x * np.sum(gamma))/(2*x-2)) - digamma((2*(n_samples -1) - x * np.sum(gamma))/(2*x-2))  + 1 + np.log(x) - x, bracket=[1e-8, 1e3])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


      converged: True
           flag: 'converged'
 function_calls: 12
     iterations: 11
           root: 0.0

NameError: name 'x' is not defined

In [467]:
root_scalar(lambda x: np.log(x) - digamma(x), bracket=[1e-10, 1e3])

ValueError: f(a) and f(b) must have different signs

In [949]:
vec = theta_normed[:, 2:3] 

In [950]:
vec.T @ vec

array([[1.]])

In [951]:
vec @ vec.T

array([[5.74629204e-170, 8.86335466e-136, 2.96242319e-121, ...,
        1.17596081e-091, 1.78109289e-091, 2.60040387e-091],
       [8.86335466e-136, 1.36712606e-101, 4.56938269e-087, ...,
        1.81385799e-057, 2.74724254e-057, 4.01098684e-057],
       [2.96242319e-121, 4.56938269e-087, 1.52723723e-072, ...,
        6.06250700e-043, 9.18218365e-043, 1.34060307e-042],
       ...,
       [1.17596081e-091, 1.81385799e-057, 6.06250700e-043, ...,
        2.40656725e-013, 3.64495125e-013, 5.32164571e-013],
       [1.78109289e-091, 2.74724254e-057, 9.18218365e-043, ...,
        3.64495125e-013, 5.52058938e-013, 8.06008607e-013],
       [2.60040387e-091, 4.01098684e-057, 1.34060307e-042, ...,
        5.32164571e-013, 8.06008607e-013, 1.17677630e-012]])

In [952]:
np.linalg.inv(vec @ vec.T)

LinAlgError: Singular matrix

In [962]:
qi

array([[-0.00205166],
       [ 0.00285809],
       [ 0.89297874],
       [-0.00780272],
       [-0.00682325],
       [ 0.0093734 ],
       [-0.00101417],
       [-0.01494169],
       [-0.00568084],
       [ 0.01432373],
       [-0.00136691],
       [-0.0168437 ]])

In [963]:
si

array([[1.        ],
       [1.        ],
       [0.99999979],
       [1.        ],
       [0.831427  ],
       [1.        ],
       [0.22601206],
       [1.        ],
       [0.50705051],
       [1.        ],
       [0.36033717],
       [1.        ]])

25000.10525077745

In [974]:
theta_normed[:, 2:3].T @ y_normed

array([[0.89297893]])

In [987]:
beta * qi**2 - si

array([[-8.94766970e-01],
       [-7.95782762e-01],
       [ 1.99343598e+04],
       [ 5.22065743e-01],
       [ 3.32497168e-01],
       [ 1.19652652e+00],
       [-2.00298425e-01],
       [ 4.58137756e+00],
       [ 2.99751120e-01],
       [ 4.12925113e+00],
       [-3.13625906e-01],
       [ 6.09278429e+00]])

In [1183]:
from DeePyMoD_SBL.SBL.robust_SBL import initialize, posterior, sparse_quality_factor, update_filter, update_noise, convergence, update_design_matrix

In [1184]:
x_points = 100
t_points = 25
x_grid, t_grid = np.meshgrid(np.linspace(-10, 10, x_points), np.linspace(0, 1, t_points), indexing='ij')
time_deriv, theta = library(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1))

In [1242]:
level = 2.0
y = time_deriv + level*np.std(time_deriv)* np.random.normal(size=time_deriv.shape)

In [1243]:
level*np.std(time_deriv)

0.5881291532252986

In [1244]:
l = 0.0
n_samples = theta.shape[0]

In [1274]:
beta, gamma, Phi, theta_normed, y_normed = initialize(theta, y)
mu, Sigma = posterior(Phi, gamma, y_normed)
si, qi = sparse_quality_factor(theta_normed, Phi, Sigma, gamma, y_normed)
beta = update_noise(n_samples, y_normed, Phi, mu)
l = update_filter(gamma, n_samples, beta, qi, si)

In [1275]:
beta

array([[3145.62848293]])

In [1276]:
l

0.9725001257798382

In [1248]:
gamma

array([   0.        ,    0.        , 5131.57916483,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ])

In [1272]:
Phi, gamma = update_design_matrix(Phi, theta_normed, gamma, si, qi, beta, l)
mu, Sigma = posterior(Phi, gamma, y_normed)
si, qi = sparse_quality_factor(theta_normed, Phi, Sigma, gamma, y_normed)
beta = update_noise(n_samples, y_normed, Phi, mu)
l = update_filter(gamma, n_samples, beta, qi, si)

In [1273]:
gamma

array([0.00000000e+00, 0.00000000e+00, 2.01807107e-08, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [1271]:
delta = (n_samples-1) / np.max(beta * qi**2 - si)
delta

4.8788041363022865

In [1252]:
2* (n_samples-1)/(np.sum(gamma)+ 2 * delta)

155.74817421857537

In [1253]:
l

155.74817421857537

In [1289]:
converged = False
nu = 1.0
while converged is False:
    Phi, gamma = update_design_matrix(Phi, theta_normed, gamma, si, qi, beta, l)
    mu, Sigma = posterior(Phi, gamma, y_normed)
    si, qi = sparse_quality_factor(theta_normed, Phi, Sigma, gamma, y_normed)
    beta = update_noise(n_samples, y_normed, Phi, mu)
    l = update_filter(gamma, nu, n_samples, beta, qi, si)
    converged = convergence(si, qi, l, beta, gamma)
    

In [1293]:
beta * Sigma

array([[6.76007636e-09]])

In [1288]:
delta = (n_samples-1)/np.max(beta*qi**2 - si)

In [1280]:
delta

4.8788041363022865

In [1297]:
np.linalg.norm(y)

32.84916265402285

In [1298]:
np.linalg.norm(theta, axis=0)

array([50.        ,  3.9956639 ,  7.35161442, 20.62151779,  5.28920353,
        1.42388416,  4.188986  , 11.14552297,  1.89564284,  0.74350554,
        2.98434239,  6.92921691])

In [1209]:
l = 2 * (n_samples - 1) / (np.sum(gamma) + 2 * delta)

In [1210]:
mu

array([[0.65209576]])

In [1212]:
beta * qi**2 - si

array([[-3.18377648e-01],
       [-3.58725624e-01],
       [ 5.38310716e+02],
       [ 2.37534964e+00],
       [ 6.47291271e+02],
       [ 8.86690105e-01],
       [ 5.32508100e+02],
       [ 6.14569559e-01],
       [ 7.03140728e+02],
       [ 2.39094715e-01],
       [ 4.41049606e+02],
       [-3.01170433e-01]])

In [1217]:
from copy import copy
from DeePyMoD_SBL.SBL.robust_SBL import dL

In [1218]:
    gamma_current = copy(gamma)
    gamma_new = ((-2 * l - si + np.sqrt(4*beta*l*qi**2+si**2)) / (2*l*si)).squeeze()

    in_basis_idx = (gamma_current != 0.0)
    out_basis_idx = (gamma_current == 0.0)
    allowed = beta * qi**2 - si >= l
    
    dL_adding = dL(gamma_new, si, qi, beta, l)
    dL_adding[in_basis_idx] = 0.0
    dL_adding[~allowed] = 0.0
    
    dL_removing = -dL(gamma_current, si, qi, beta, l)
    dL_removing[out_basis_idx] = 0.0
    dL_removing[allowed] = 0.0
    
    dL_updating = dL(gamma_new, si, qi, beta, l) -dL(gamma_current, si, qi, beta, l)
    dL_updating[out_basis_idx] = 0.0
    dL_updating[~allowed] = 0.0
    
    dL_complete = np.concatenate((dL_adding, dL_updating, dL_removing), axis=1)
    idx, action = np.unravel_index(np.argmax(dL_complete), dL_complete.shape)

In [1219]:
dL_complete

array([[   0.        ,    0.        , -771.63412366],
       [   0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ]])

In [1220]:
beta * qi**2 - si

array([[-3.18377648e-01],
       [-3.58725624e-01],
       [ 5.38310716e+02],
       [ 2.37534964e+00],
       [ 6.47291271e+02],
       [ 8.86690105e-01],
       [ 5.32508100e+02],
       [ 6.14569559e-01],
       [ 7.03140728e+02],
       [ 2.39094715e-01],
       [ 4.41049606e+02],
       [-3.01170433e-01]])

In [1221]:
l

814.9443784067988

# Final testing before rewriting

In [2]:
from DeePyMoD_SBL.SBL.robust_SBL import robust_SBL

In [6]:
x_points = 100
t_points = 25
x_grid, t_grid = np.meshgrid(np.linspace(-10, 10, x_points), np.linspace(0, 1, t_points), indexing='ij')
time_deriv, theta = library(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1))

In [7]:
level = 2.0
y = time_deriv + level*np.std(time_deriv)* np.random.normal(size=time_deriv.shape)

In [8]:
robust_SBL(theta, y)

/home/working/src/DeePyMoD_SBL/SBL/robust_SBL.py:45: RuntimeWarning: invalid value encountered in true_divide
  gamma_new = ((-2 * l - si + np.sqrt(4*beta*l*qi**2+si**2)) / (2*l*si)).squeeze()
/home/working/src/DeePyMoD_SBL/SBL/robust_SBL.py:101: RuntimeWarning: invalid value encountered in less_equal
  if np.all((beta * qi**2 - si)[gamma == 0.0] <=l): # if all elements not in model shouldnt be in model
/home/working/src/DeePyMoD_SBL/SBL/robust_SBL.py:49: RuntimeWarning: invalid value encountered in greater_equal
  allowed = beta * qi**2 - si >= l


array([0.00000000e+00, 0.00000000e+00, 2.28593963e-07, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

# testing SBL on burgers

In [42]:
from DeePyMoD_SBL.SBL import SBL
from DeePyMoD_SBL.data import Burgers

In [43]:
x_points = 100
t_points = 25
x_grid, t_grid = np.meshgrid(np.linspace(-10, 10, x_points), np.linspace(1e-4, 1, t_points), indexing='ij')

In [44]:
burgers = Burgers(0.1, 1)
theta = burgers.library(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1))
time_deriv = burgers.time_deriv(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1))

In [56]:
level = 0.5
y = time_deriv + level*np.std(time_deriv)* np.random.normal(size=time_deriv.shape)
SBL(theta, y)

(array([[        inf],
        [        inf],
        [93.40636533],
        [        inf],
        [        inf],
        [ 1.00349894],
        [        inf],
        [        inf],
        [        inf],
        [        inf],
        [        inf],
        [        inf]]), array([[ 0.10164746],
        [-0.99818633]]), array([[8.36414409e-07, 2.50056541e-07],
        [2.50056541e-07, 1.37305888e-04]]), array([[0.38583313]]))

# Testing robust SBL

In [57]:
from DeePyMoD_SBL.SBL import robust_SBL
from DeePyMoD_SBL.data import Burgers

In [58]:
x_points = 100
t_points = 25
x_grid, t_grid = np.meshgrid(np.linspace(-10, 10, x_points), np.linspace(1e-4, 1, t_points), indexing='ij')

In [59]:
burgers = Burgers(0.1, 1)
theta = burgers.library(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1))
time_deriv = burgers.time_deriv(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1))

In [60]:
level = 0.5
y = time_deriv + level*np.std(time_deriv)* np.random.normal(size=time_deriv.shape)

In [61]:
robust_SBL(theta, y)

/home/working/src/DeePyMoD_SBL/SBL/robust_SBL.py:45: RuntimeWarning: invalid value encountered in true_divide
  gamma_new = ((-2 * l - si + np.sqrt(4*beta*l*qi**2+si**2)) / (2*l*si)).squeeze()
/home/working/src/DeePyMoD_SBL/SBL/robust_SBL.py:101: RuntimeWarning: invalid value encountered in less_equal
  if np.all((beta * qi**2 - si)[gamma == 0.0] <=l): # if all elements not in model shouldnt be in model
/home/working/src/DeePyMoD_SBL/SBL/robust_SBL.py:49: RuntimeWarning: invalid value encountered in greater_equal
  allowed = beta * qi**2 - si >= l


array([0.00000000e+00, 0.00000000e+00, 6.16415096e-07, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [89]:
from DeePyMoD_SBL.SBL.robust_SBL import initialize, posterior, sparse_quality_factor, update_filter, update_noise, convergence, update_design_matrix, dL

In [94]:
beta, gamma, Phi, theta_normed, y_normed = initialize(theta, y)
nu = 1.0
l = 0.0
n_samples = theta_normed.shape[0]


In [95]:
mu, Sigma = posterior(Phi, gamma, y_normed)
si, qi = sparse_quality_factor(theta_normed, Phi, Sigma, gamma, y_normed)
#beta = update_noise(n_samples, y_normed, Phi, mu)
l = update_filter(gamma, nu, n_samples, beta, qi, si)

In [96]:
beta*qi**2 - si

array([[2.66815712e+00],
       [4.44257170e+03],
       [1.27700253e+04],
       [1.22905275e+02],
       [1.46940384e+02],
       [7.18905977e+03],
       [3.77346263e+02],
       [2.53786339e+03],
       [6.69265239e+02],
       [6.54442824e+03],
       [6.03835257e+02],
       [4.12779209e+03]])

In [103]:
converged = False
while not converged:
        Phi, gamma = update_design_matrix(Phi, theta_normed, gamma, si, qi, beta, l)
        mu, Sigma = posterior(Phi, gamma, y_normed)
        si, qi = sparse_quality_factor(theta_normed, Phi, Sigma, gamma, y_normed)
        #beta = update_noise(n_samples, y_normed, Phi, mu)
        l = update_filter(gamma, nu, n_samples, beta, qi, si)
        #converged = convergence(si, qi, l, beta, gamma)

/home/working/src/DeePyMoD_SBL/SBL/robust_SBL.py:47: RuntimeWarning: invalid value encountered in sqrt
  in_basis_idx = (gamma_current != 0.0)
/home/working/src/DeePyMoD_SBL/SBL/robust_SBL.py:51: RuntimeWarning: invalid value encountered in greater_equal
  dL_adding = dL(gamma_new, si, qi, beta, l)


KeyboardInterrupt: 

In [104]:
l

1263.9479294209884

In [114]:
beta*qi**2 - si

array([[-1.63539147e-01],
       [ 2.14004740e+03],
       [ 1.05668107e+04],
       [ 4.58575327e+02],
       [ 1.25202949e+02],
       [ 4.11657394e+03],
       [ 4.24115253e+02],
       [ 1.00469422e+02],
       [ 5.04381413e+02],
       [ 1.05154618e+03],
       [ 9.46049375e+01],
       [ 7.83829145e+02]])

In [111]:
beta*qi**2 - si > l_new

array([[False],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False]])

In [106]:
gamma

array([0.        , 0.4982793 , 1.97512692, 0.        , 0.        ,
       1.02022304, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])

In [107]:
delta = (n_samples - 1 + nu) / np.max(beta * qi**2 - si)
2 * (n_samples - 1 + nu) / (np.sum(gamma) + 2 * delta)

1260.4589962548564

In [108]:
delta

0.2365898352801321

In [110]:
l_new = 2 * (n_samples - 1 + nu) / (np.sum(gamma))

In [87]:
from copy import copy

In [92]:
    gamma_current = copy(gamma)
    gamma_new = ((-2 * l - si + np.sqrt(4*beta*l*qi**2+si**2)) / (2*l*si)).squeeze()

    in_basis_idx = (gamma_current != 0.0)
    out_basis_idx = (gamma_current == 0.0)
    allowed = beta * qi**2 - si >= l

    dL_adding = dL(gamma_new, si, qi, beta, l)
    dL_adding[in_basis_idx] = 0.0
    #dL_adding[~allowed] = 0.0

    dL_removing = -dL(gamma_current, si, qi, beta, l)
    dL_removing[out_basis_idx] = 0.0
    #dL_removing[allowed] = 0.0

    dL_updating = dL(gamma_new, si, qi, beta, l) -dL(gamma_current, si, qi, beta, l)
    dL_updating[out_basis_idx] = 0.0
    #dL_updating[~allowed] = 0.0

    dL_complete = np.concatenate((dL_adding, dL_updating, dL_removing), axis=1)
    idx, action = np.unravel_index(np.argmax(dL_complete), dL_complete.shape)

In [93]:
dL_complete

array([[1.27479248e+03, 0.00000000e+00, 0.00000000e+00],
       [4.11358170e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 3.69208352e-10, 3.61315707e-10],
       [1.64420894e+02, 0.00000000e+00, 0.00000000e+00],
       [6.98514852e+02, 0.00000000e+00, 0.00000000e+00],
       [6.54521618e+01, 0.00000000e+00, 0.00000000e+00],
       [1.81408007e+02, 0.00000000e+00, 0.00000000e+00],
       [1.05279514e+03, 0.00000000e+00, 0.00000000e+00],
       [3.07073680e+02, 0.00000000e+00, 0.00000000e+00],
       [1.74096588e+02, 0.00000000e+00, 0.00000000e+00],
       [5.70402199e+02, 0.00000000e+00, 0.00000000e+00],
       [2.82590185e+02, 0.00000000e+00, 0.00000000e+00]])